In [1]:
import sys
sys.path.append("..")

import torch
import torch.optim as optim

import matplotlib.pyplot as plt

from datasets import *
from models.ad_models import *

torch.manual_seed(1234)

/home/antonxue/lib/miniconda3/envs/arpro/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SyntaxError: invalid syntax (vision.py, line 45)

In [ ]:
model = VaeADModel()
state_dict = torch.load("../_dump/vae_mvtec_transistor_best.pt")["model_state_dict"]
model.load_state_dict(state_dict)
model.eval()
;

In [ ]:
torch.manual_seed(1234)
dataloader = get_ad_dataloader("mvtec", "vae", batch_size=8, category="transistor", split="test", normalize_image=False)
for batch in dataloader:
    break

In [ ]:
batch["label"]

In [ ]:
with torch.no_grad():
    x = batch["image"]
    out = model(2*x - 1)

In [ ]:
score = out.score
thresh = out.alpha.flatten(1).quantile(0.99, dim=1)
batch["label"], score, thresh

In [ ]:
plt.clf()

fig, ax = plt.subplots(4,8, figsize=(12, 8))
for i in range(8):
    xi = x[i].detach()
    xhati = (out.others["x_recon"][i].detach()*0.5 + 0.5).clamp(0,1)
    alphai = out.alpha[i].abs().max(dim=0).values.detach()
    alphai_big = alphai > thresh[i]
    ax[0,i].imshow(xi.numpy().transpose(1,2,0))
    ax[1,i].imshow(xhati.numpy().transpose(1,2,0))
    ax[2,i].imshow(alphai.numpy())
    ax[3,i].imshow(alphai_big.numpy())
    